In [137]:
#%%

%load_ext autoreload
%autoreload 2

import numpy as np
from planning_graph.planning_graph import PlanningGraph, NoOpAction
from pddlpy.pddl import Operator


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [172]:
from planning_graph.planning_graph import PlanningGraph
from planning_graph.planning_graph_planner import GraphPlanner


planning_graph = PlanningGraph('domain/dock-worker-robot-domain.pddl', 
                               'domain/dock-worker-robot-problem.pddl')

graph = planning_graph.create(max_num_of_levels=10)
goal = planning_graph.goal
graph_planner = GraphPlanner()
layered_plan = graph_planner.plan(graph, goal)
layered_plan[2]

Plan object. [NoOp Action. Preconds={('loaded', 'robr', 'conta')}, Effects={('loaded', 'robr', 'conta')}, <pddlpy.pddl.Operator object at 0x00000106B06E0550>, <pddlpy.pddl.Operator object at 0x00000106B06E00D0>, NoOp Action. Preconds={('loaded', 'robq', 'contb')}, Effects={('loaded', 'robq', 'contb')}]

In [138]:
# F:\@uni_life\@COMP3770\@project\pddl-plan-database\ipc\benchmarks\ipc-2018\domains\agricola is out of range
# F:\@uni_life\@COMP3770\@project\pddl-plan-database\ipc\benchmarks\ipc-2008\domains\elevator is out of range
# F:\@uni_life\@COMP3770\@project\pddl-plan-database\htg\benchmarks\blocksworld-large-simple is taking too long
# F:\@uni_life\@COMP3770\@project\pddl-plan-database\hgn\benchmarks\blocks works fine
# F:\@uni_life\@COMP3770\@project\pddl-plan-database\hgn\benchmarks\zenotravel


planning_graph = PlanningGraph('domain/blocks/domain.pddl', 
                               'domain/blocks/blocks/blocks5/task01.pddl',
                               visualize=True)

graph = planning_graph.create(max_num_of_levels=10)
states = graph.prop


In [139]:
# test0 is the default test
# test1 is the blocks from bgn
# test2 is the zenotravel from bgn
graph.visualize_png("test2-s0.png")

In [140]:
graph_1 = planning_graph.create_with_state(states[1], max_num_of_levels=10)
graph_1.visualize_png("test2-s1.png")

In [141]:
graph_2 = planning_graph.create_with_state(states[2], max_num_of_levels=10)
graph_2.visualize_png("test2-s2.png")

In [147]:
act_schema = np.array(list(planning_graph._planning_problem._domain_problem.operators()))
type(act_schema[0])

numpy.str_

In [230]:
a=[1,2,3]
for i in a[1:3]:
    print(i)

2
3


In [262]:
def generate_feature_vec(planning_graph, state, max_level):
    # draw graph first
    graph = planning_graph.create_with_state(state, max_num_of_levels=max_level)
    graph.visualize_png("test_feature_vec.png")
    
    # get action schema and output list
    act_schema = np.array(list(planning_graph._planning_problem._domain_problem.operators()))
    n = len(act_schema)
    feature_vec = np.zeros(n+2*n**2+3)
    
    # extract linear feature
    counter = np.zeros(n)
    for act_layer in graph.act.values():
        if act_layer is not None:
            for a in act_layer:
                 if not isinstance(a, NoOpAction):
                        counter[act_schema == a.operator_name] += 1
    feature_vec[0:n] = counter
    
    # extract pair-wise feature
    def to_index(n, index_a1, index_a2, adder):
        return n+2*(n*index_a1+index_a2)+adder
    
    act_layers = list(graph.act.values())
    pre = {}
    eff_pos = {}
#     eff_neg = {}
    
    def append_to_dict(a, pre, eff_pos):
        for p in a.precondition_pos:
            current = pre.get(p)
            if current is None:
                current = [a]
            else:
                current.append(a)
            pre[p] = current
            
        for p in a.effect_pos:
            current = eff_pos.get(p)
            if current is None:
                current = [a]
            else:
                current.append(a)
            eff_pos[p] = current
            
#         for p in a.effect_neg:
#             current = effect_neg.get(p)
#             if current is None:
#                 current = [name]
#             else:
#                 current.append(name)
#             effect_neg[p] = current
#         return pre, eff_pos, eff_neg
            return pre, eff_pos
    
    # add pre and eff into the storage for the first layer
    for a in act_layers[1]:
        if not isinstance(a, NoOpAction):
            pre, eff_pos = append_to_dict(a, pre, eff_pos)
        
    for i in range(2,len(act_layers)): 
        act_layer = act_layers[i]
        # update feature vector
        for a2 in act_layer:
            if not isinstance(a2, NoOpAction):
                print("pre:",a2.precondition_pos,"eff:", a2.effect_pos)
                # count for num of occurances
                s1 = set()
                s2 = set()

                for p in a2.precondition_pos:
                    current = eff_pos.get(p)
                    if current is not None:
                        for a1 in current:
                            s1.add(a1)

                for p in a2.effect_pos:
                    current = pre.get(p)
                    if current is not None:
                        for a1 in current:
                            s2.add(a1)
                # add index to feature_vec based on set:
                index_a2 = int(np.where(a2.operator_name == act_schema)[0])
                for a1 in s1:
                    index_a1 = int(np.where(a1.operator_name == act_schema)[0])
                    feature_vec[to_index(n, index_a1, index_a2,0)]+=1
                    print(a1.effect_pos," s1 ")
                for a1 in s2:
                    index_a1 = int(np.where(a1.operator_name == act_schema)[0])
                    feature_vec[to_index(n, index_a1, index_a2,1)]+=1
                    print(a1.precondition_pos, " s2")
                print("finish\n")

        # update pre and eff_pos
        for a2 in act_layer:
            if not isinstance(a2, NoOpAction):
                pre, eff_pos = append_to_dict(a2, pre, eff_pos)
                
                 
    
    # add heuristic value, number of layers and number of unsatisfied goals   
    
    return feature_vec
            
        

In [263]:
planning_graph_f = PlanningGraph('domain/blocks/domain.pddl', 
                               'domain/blocks/blocks/blocks4/task01.pddl',
                               visualize=True)

graph_f = planning_graph_f.create(max_num_of_levels=10)
states_f = graph_f.prop

f_v = generate_feature_vec(planning_graph_f, states_f[1], 10)

pre: {('clear', 'b4'), ('on-table', 'b4'), ('handempty',)} eff: {('holding', 'b4')}
{('clear', 'b4'), ('on', 'b4', 'b3'), ('handempty',)}  s1 
{('clear', 'b4'), ('handempty',), ('on', 'b4', 'b4')}  s1 
{('clear', 'b4'), ('on-table', 'b4'), ('handempty',)}  s1 
{('clear', 'b4'), ('on', 'b4', 'b1'), ('handempty',)}  s1 
{('clear', 'b3'), ('holding', 'b4')}  s2
{('clear', 'b4'), ('holding', 'b4')}  s2
{('holding', 'b4')}  s2
{('holding', 'b4'), ('clear', 'b1')}  s2
finish

pre: {('clear', 'b3'), ('on-table', 'b3'), ('handempty',)} eff: {('holding', 'b3')}
{('clear', 'b3'), ('on-table', 'b3'), ('handempty',)}  s1 
{('clear', 'b3'), ('on', 'b3', 'b3'), ('handempty',)}  s1 
{('holding', 'b3')}  s2
{('clear', 'b4'), ('holding', 'b3')}  s2
{('holding', 'b3'), ('clear', 'b3')}  s2
{('holding', 'b3'), ('clear', 'b1')}  s2
finish

pre: {('holding', 'b4')} eff: {('clear', 'b4'), ('on-table', 'b4'), ('handempty',)}
{('holding', 'b4'), ('clear', 'b1')}  s1 
{('clear', 'b4'), ('on', 'b4', 'b1'), ('ha

In [264]:
print(f_v)

[ 11.  12.  44.  32.   0.   0.   7.  25.  27.  95.   0.  20.   8.   8.
   0.   0.  20.   0.  22.  29.  23.  29.   0.  26.  51.  99.  91. 157.
   7.   0.  13.  71.  76. 271.  28.  57.   0.   0.   0.]
